In [5]:
from document_constrained_generation_causal_qwen import IndexBasedLogitsProcessor
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList
from index import FMIndex
import torch

In [2]:
index_dir = 'data/enwiki.qwen3_8b.fm_index'

In [3]:
index = FMIndex.load(index_dir)

In [6]:
model_name = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
from document_constrained_generation_causal_qwen import IndexBasedLogitsProcessor

In [9]:
prompt = f"What is the greek goddess of beauty? /no_think"
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
input_ids = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
out_ids = model.generate(**input_ids, max_new_tokens=50, min_new_tokens=10, do_sample=False, num_beams=2,
                              logits_processor=LogitsProcessorList([IndexBasedLogitsProcessor(
                                    num_beams=2,
                                    min_new_tokens=10,
                                    index=index,
                                    model_name=model_name,
                                    pad_token_id=model.config.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    force_decoding_from=None,
                                    stop_at_count=0,
                                    always_allow_eos=True,
                                    forced_bos_token_id=None,
                                    length_reward_factor=1
                                )]),
                              temperature=None, top_p=None, top_k=None)

In [ ]:
gen_output = tokenizer.batch_decode(out_ids, skip_special_tokens=True,
                                         clean_up_tokenization_spaces=False)
# print(out_ids)
print(gen_output)